---
# Text Classification for Toxic Speech Detection 
    
## Alex Newhouse 
---


### Problem Statement

One of the most challenging, and most important, problems facing online social platforms is content moderation. In order to foster a supportive, inclusive, and safe community, social platforms must implement sophisticated measures to mitigate abusive behaviors. In general, these measures can range from hiring large numbers of content moderators, who review and make decisions on content full-time, to advanced AI models that remove violative content automatically.

However, an evolution in adversarial behavior and the nuances of extreme ideologies and language have caused an arms race between content moderation teams and toxic actors. Technology companies have increasingly sought out more advanced techniques, including expert-in-the-loop AI training systems as well as building structures to allow content moderators to work _alongside_ AI. I am involved in one such effort for my job, [providing subject-matter qualitative expertise](https://www.middlebury.edu/institute/academics/centers-initiatives/ctec/news/ctec-partners-spectrum-labs-research-language-extremism) to the development of model thresholds and training data for Spectrum Labs, a startup that develops AI technology for toxic content detection.

In 2019, Jigsaw, the Google-owned thinktank that specializes in anti-toxicity and anti-disinformation technology, launched a Kaggle competition based on a dataset of toxic social posts compiled by Civil Comments and the University of California - Berkeley. [1] Jigsaw aimed to solicit novel ideas for building AI technologies that could simultaneously 1) detect toxic comments and 2) reduce the implicit anti-LGBTQ/anti-minority biases that AI models can acquire when trained on large amounts of toxic speech. The winners of the competition largely used LSTM- and transformer-based architectures, [2] which are complex neural networks that are currently state-of-the-art in a variety of language tasks. [3]

In this paper, I investigate the possibility of achieving high levels of accuracy, precision, and recall using methods that are much less computationally intensive than the Kaggle winners. While LSTMs and Transformers are useful for large technology companies, many more small platforms and forums rely exclusively on volunteer moderators due to resource constraints. Smaller, more nimble models are more likely to be useful to these communities. As such, I build a number of classifiers, ranging from simple - a basic logistic regression using the bag-of-words method - to more complex - neural network classifiers with preprocessing. I assess these models based on several metrics, including Jigsaw's own variation on AUC that takes into account implicit demographic bias in models. [4]

### Methodology

My goal is to predict the level of toxicity of an online post given only the text of that post. This requires using numeric representations of text data as the independent variables, and I use different common methods for doing this: bag-of-words (BOW) vector representations and TF-IDF BOW representations. In order to improve performance, I also test pre-processing the dataset by removing stop words, uncensoring toxic content, and lemmatizing text. I further test weighting the positive (toxic) class, given that the dataset is heavily imbalanced. 

I focus on three types of classification models: Naive Bayes, Logistic Regression, and Multi-Layered Perceptron. I use grid-searching and k-fold cross-validation for model building. Finally, I compare across the models using common classification metrics: precision, recall, accuracy, AUC, and F1 score, as well as Jigsaw's own bias-sensitive score.

### Data Description

The dataset I use was collected by Civil Comments and UC Berkeley, and published by Jigsaw during the Kaggle competition. It consists of 1.8 million comments and 46 features, including the "target" column, which is a toxicitiy score from 0 to 1, and the "comment text" column. These two columns are the most important for predicting toxicity, and I drop the other 44 features for the purposes of this project. These columns are other characteristics of the text data, such as markers for different types of toxicity, metrics for "likes", and some aggregate measures of the behavior of the researchers who labeled the dataset manually. 

![title](words_chart.jpg)

The data spans the time period from Sept. 29, 2015 to Nov. 11, 2017. The comments were all posted through the Civil Comments plugin, which was used between 2015-2017 by some news sites and other online communities that needed comment functionality. In accordance with Jigsaw's instructions on the Kaggle competition, I assign the positive label to comments that earned .5 or higher toxicity score, and the negative label to those that have a sub-.5 score. 

Not surprisingly, this dataset is heavily imbalanced. 92% of comments are assigned the negative (non-toxic) label, while only 8% are toxic. This presents a classification challenge, as imbalanced data will often result in classifiers that overemphasize the majority class. In this investigation, I frequently weight the classes in order to deal with the issues of the imbalanced data. 

![title](final_report_plot.jpg)

### Data Processing

_Basic Data Preparation_
Because of the size of the dataset, I split the data 70% training and 30% testing, and I also use 5-fold cross-validation on the data. Although CV is a technique to deal with overfitting on training data, out of caution for random effects that may occur, I also test the models on the holdout data. 

_Numeric text representations_

In order to run classification algorithms on text data, I must first convert the comment text into a numeric representation. The simplest way to do this is called the "Bag-of-Words" method: the vocabulary of unique words in the dataset, across all comments, are assigned a number, and then each comment is represented by the sequence of numbers. In this dataset, there are 264,195 words in the vocabulary, and so a tokenized comment with 9 unique words would be represented as a sparse vector of 264,186 zeros and 9 ones. 

A more sophisticated way of capturing context and differences between toxic and non-toxic comments is to use Term Frequency-Inverse Document Frequency, or TF-IDF. These weights are given by the equation: 

$$ w_{x,y} = \text{tf}_{x,y} * log(\frac{N}{\text{df}_x}) $$

where $\text{tf}_{x,y}$ is the frequency of term $x$ in document $y$, $N$ is the number of documents, and $\text{df}_x$ is the number of documents that contain term $x$. In essence, TF-IDF is designed to produce weights that capture instances where a term is used frequently in a single document but infrequently over the entire document set. 

_Normalization_

Especially in the simpler numeric representations, getting rid of some of the irrelevant differences between closely related words is important. For example, a straightforward classifier should not receive two distinct representations for the words "write" and "writing". In addition, stop words - basic conjunctions, articles, and other neutral words - make vectors larger without granting any more knowledge to the model. In my pre-processing steps, I remove normal English stop words and "lemmatize", or reduce words to their stems, the dataset. I also un-censor slurs and swear words that were occasionally censored, in order to pick up on words that are strongly related to toxicity.   

### Evaluation

The first and most obvious concern when building a classifier on this dataset is the class imbalance. As such, a straightforward measure of accuracy is not particularly useful, as the number of positives is too low to significantly reduce the score, even if the True Positive Rate is low. The AUC score and the F1 score are much more useful, as they take into account the relative goodness of each class's predictions.

The official Kaggle competition used a normal AUC score, but it also introduced a number of novel metrics in order to assess _bias_ (in the sociological sense, not in the ML sense) in model predictions. These identity-wise AUC scores give a sense of how much a model may be unfairly targeting a specific demographic. 

These metrics are composed of several parts:

_Bias AUCs_

Jigsaw gives two different types of Bias AUC scores, "Background Positive, Subgroup Negative" and "Background Negative, Subgroup Positive". A low BPSN score indicates a model that confuses non-toxic mentions of an identity with toxic ones, meaning that it unfairly classifies those mentions as toxic. A low BNSP score indicates the opposite. 

_Generalized Mean of Bias AUCs_

These individual scores are combined into a single metric, given by $$ M_p(m_s) = (\frac{1}{N}\sum_{s=1}{N}m_s^p)^{\frac{1}{p}}$$

where $$M_p = \text{ the p-th power-mean function, } m_s = \text{ the bias metric for subgroup s, } N = \text{ the number of subgroups.} $$

This is then merged with the overal AUC score using the following: $$ score = w_0 AUC_{overall} + \sum_{a=1}^A w_aM_p(m_{s,a})$$

Jigsaw sets the weighting w per submetric at .25. I use metrics code provided by Jigsaw during the competition. [5]

### Results

In the simplest and least-processed scenario, where the comments are one-hot encoded vectors and I performed no pre-processing on them, the simplest models perform surprsiingly well. The logistic regression model in particular - even unweighted by class - achieves an AUC score above .90. Relative to the more complex models, including the multi-layer perceptron and the boosted algorithms, the logistic regression model performs much better on the holdout data in terms of precision and recall. As a result of the imbalanced classes, higher scores on precision and recall are important in boosting F1 and AUC scores. 

The ability of a logistic regression model to perform this well on an unprocessed dataset is encouraging, as these models are highly flexible, can produce predictions with low computational power, and are capable of revealing feature importance metrics. It outperforms the slower, more intensive random forest and perceptron models, too.

I find that transforming the text into TF-IDF vectors can boost the performance of these simple classifiers with no other changes necessary. The same logistic regression model earns a .94 AUC score with TF-IDF, indicating a significant bump in its ability to deal with the positive (minority) class. Adding in additional normalization - namely, lemmatizing and uncensoring words - has a minor (at most) positive impact on AUC scores. However, lemmatization and uncensoring reduced the total feature space by about 150,000 unique words, meaning that the models are still performing better while being trained on a more compressed dataset (which also resulted in a reduction in training times).

The bias-sensitive AUC score poses a unique challenge, and the results bear out that it is measuring something quite different than the straightforward AUC. In fact, the Naive Bayes algorithm scores the best on this measure when using simple vectorization, even though its AUC score is lower. However, the Naive Bayes model loses most of its capability when run on the TF-IDF vectorized data, and its predictions are no better than random guesses. The rest of the models show little improvement in bias AUC using TF-IDF, indicating that the problem likely hinges more on subtle linguistic context that the TF-IDF score cannot pick up.

Compared with the top Kaggle scores, which are above .94 on Jigsaw's Bias AUC metric, these models are far worse. Much more feature engineering and other anti-bias interventions should be studied in the future to determine whether or not simple machine learning models can _ever_ accomplish non-biased predictions like transformers can. But their success on the straightforward AUC scores indicates that they may still be useful for hate speech detection. 

![title](results.png)


#### Bias AUC scores:
_Simple Vectorization:_
* Logistic Regression - **.71**
* Naive Bayes - **.73**
* Multilayered Perceptron - **.72**

_TF-IDF Vectorization:_
* Logistic Regression - **.71**
* Naive Bayes - **.51**
* Multilayered Perceptron - **.50**

### Conclusion

After training dozens of models on the toxic content dataset, it became clear that low-weight machine learning models cannot compete with highly complex, large transformers and LSTMs on text classification _when demographic bias is taken into account_. There is limited capacity of basic learning algorithms, like logistic regression, to pick up on the nuances of toxic vs. non-toxic data, especially when only trained on superficial, Bag-of-Words text vectors. One of the main advantages of neural nets over my experiments is that they use contextually sensitive word embeddings, which not only capture different words with similar meanings, but also capture the difference between words that are spelled the same but have different meanings. The training data I used is not capable of such nuance.

The positive, however, is that logistic regression and even Naive Bayes performed surprisingly well on more traditional measures of straightforward AUC and F1. Both scoring greater than .9 on AUC, this indicates that even simple, contextless data fed into quick-to-train models can be extremely useful. I envision small, flexible models like these to be deployed to assist moderators and forum admins on small-scale or non-commercial web communities, where they cannot rely on transformer models like the large technology companies can. With training rates measured in minutes and inference in seconds, these simple models - especially Logistic Regression - have proven that they are still more than useful for assisting humans with heavy content moderation workloads.

**_References_**

[1] https://medium.com/@aja_15265/saying-goodbye-to-civil-comments-41859d3a2b1d

[2] https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/leaderboard

[3] https://arxiv.org/abs/1706.03762

[4] https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/overview/evaluation 

[5] https://www.kaggle.com/dborkan/benchmark-kernel

### Code to run models

In [1]:
## Imports

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score,roc_curve, roc_auc_score, precision_score, recall_score, precision_recall_curve
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.base import BaseEstimator
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

In [11]:
## Code from Kaggle Benchmark Kernel: https://www.kaggle.com/dborkan/benchmark-kernel
## This code calculates the overall AUC score that takes into account identity-specific bias scores.

MODEL_NAME = 'lr'
TOXICITY_COLUMN = 'label'
TEXT_COLUMN = 'comment_text'
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']

def convert_to_bool(df, col_name):
    df[col_name] = np.where(df[col_name] >= 0.5, True, False)
    
def convert_dataframe_to_bool(df):
    bool_df = df.copy()
    for col in ['label'] + identity_columns:
        convert_to_bool(bool_df, col)
    return bool_df

SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive

def compute_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan

def compute_subgroup_auc(df, subgroup, label, model_name):
    subgroup_examples = df[df[subgroup]]
    return compute_auc(subgroup_examples[label], subgroup_examples[model_name])

def compute_bpsn_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
    subgroup_negative_examples = df[df[subgroup] & ~df[label]]
    non_subgroup_positive_examples = df[~df[subgroup] & df[label]]
    examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
    return compute_auc(examples[label], examples[model_name])

def compute_bnsp_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
    subgroup_positive_examples = df[df[subgroup] & df[label]]
    non_subgroup_negative_examples = df[~df[subgroup] & ~df[label]]
    examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
    return compute_auc(examples[label], examples[model_name])

def compute_bias_metrics_for_model(dataset,
                                   subgroups,
                                   model,
                                   label_col,
                                   include_asegs=False):
    """Computes per-subgroup metrics for all subgroups and one model."""
    records = []
    for subgroup in subgroups:
        record = {
            'subgroup': subgroup,
            'subgroup_size': len(dataset[dataset[subgroup]])
        }
        record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup, label_col, model)
        record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup, label_col, model)
        record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup, label_col, model)
        records.append(record)
    return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)

def calculate_overall_auc(df, model_name):
    true_labels = df[TOXICITY_COLUMN]
    predicted_labels = df[model_name]
    return metrics.roc_auc_score(true_labels, predicted_labels)

def power_mean(series, p):
    total = sum(np.power(series, p))
    return np.power(total / len(series), 1 / p)

def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
    bias_score = np.average([
        power_mean(bias_df[SUBGROUP_AUC], POWER),
        power_mean(bias_df[BPSN_AUC], POWER),
        power_mean(bias_df[BNSP_AUC], POWER)
    ])
    return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)
    


In [12]:
## Read in data and split it into train/test arrays.

df = pd.read_csv("train.csv")
df.loc[df['target'] >= .5, 'label']=1
df.loc[df['target'] < .5, 'label'] =0
df_sub = df
df_x = np.array(df_sub['comment_text']).reshape(df_sub['comment_text'].shape[0],)
df_y = np.array(df_sub['label']).reshape(df_sub['label'].shape[0],)
train_x, test_x, train_y, test_y = train_test_split(df_x, df_y, test_size = .3, random_state = 123)

In [ ]:
## Set up bias scoring training loop: Vectorize text data, list the models to run through, then use GridSearchCV on each model. Compute bias metrics and print.

count_vect = CountVectorizer()
tfidf_vect = TfidfVectorizer(strip_accents = "unicode", stop_words = "english")
train_x_count = count_vect.fit_transform(train_x)
train_x_tf = tfidf_vect.fit_transform(train_x)
test_x_count = count_vect.transform(test_x)
test_x_tf = tfidf_vect.transform(test_x)
models = [LogisticRegression(max_iter = 5000), MultinomialNB(alpha=1), 
          MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1), RandomForestClassifier()]
print("Entering Loop")
for i in models:
    clf = GridSearchCV(i, param_grid={}, refit="f1", scoring = ["accuracy", 'precision', 'recall', "f1", 'roc_auc'], cv=5, n_jobs=-1, return_train_score=False, verbose = 3).fit(train_x_count, train_y)
    preds = clf.predict(test_x_count)
    validate_df = pd.DataFrame({"comment_text": test_x, "label": test_y, "lr": preds})
    validate_df = validate_df.merge(df_sub.drop(['label'], axis = 1), "inner", on = "comment_text")
    validate_df = convert_dataframe_to_bool(validate_df)
    bias_metrics_df = compute_bias_metrics_for_model(validate_df, identity_columns, MODEL_NAME, TOXICITY_COLUMN)
    print('Normal metrics for model ', i, " are: ")
    pd.DataFrame(clf.cv_results_)[["mean_test_accuracy","mean_test_precision", "mean_test_recall", "mean_test_f1", "mean_test_roc_auc"]]
    print("Bias AUC Score for model ", i, " is: ", get_final_metric(bias_metrics_df, calculate_overall_auc(validate_df, MODEL_NAME)))
    
    clf_tf = GridSearchCV(i, param_grid={}, refit="f1", scoring = ["accuracy", 'precision', 'recall', "f1", 'roc_auc'], cv=5, n_jobs=-1, return_train_score=False, verbose = 3).fit(train_x_tf, train_y)
    preds = clf_tf.predict(test_x_tf)
    validate_df = pd.DataFrame({"comment_text": test_x, "label": test_y, "lr": preds})
    validate_df = validate_df.merge(df_sub.drop(['label'], axis = 1), "inner", on = "comment_text")
    validate_df = convert_dataframe_to_bool(validate_df)
    bias_metrics_df = compute_bias_metrics_for_model(validate_df, identity_columns, MODEL_NAME, TOXICITY_COLUMN)
    print('Normal metrics for tf_idf model ', i, " are: ")
    pd.DataFrame(clf_tf.cv_results_)[["mean_test_accuracy","mean_test_precision", "mean_test_recall", "mean_test_f1", "mean_test_roc_auc"]]
    print("Bias AUC Score for tf_idf model ", i, " is: ", get_final_metric(bias_metrics_df, calculate_overall_auc(validate_df, MODEL_NAME)))
    

In [4]:
## Different way to run multiple models involves building a "Classifier Switch" class that lets
## us pass models as parameters in scikit-learn's GridSearchCV.

class classifier_switch(BaseEstimator):

    def __init__(
        self, 
        estimator = LogisticRegression(max_iter = 5000),
    ):
        self.estimator = estimator


    def fit(self, X, y=None, **kwargs):
        self.estimator.fit(X, y)
        return self


    def predict(self, X, y=None):
        return self.estimator.predict(X)


    def predict_proba(self, X):
        return self.estimator.predict_proba(X)


    def score(self, X, y):
        return self.estimator.score(X, y)

In [12]:
### Find censored comments

# import regex as re
# from collections import Counter
# censored = df['comment_text'].str.extractall(r"([a-zA-Z]+\*[a-zA-Z]+)")
# censored = [x.lower() for x in list(censored[0])]

In [13]:
### Uncensor slurs and swear words [NOTE: EXTREMELY HATEFUL AND TOXIC WORDS BELOW]







import regex as re
from nltk.stem import SnowballStemmer
censored = [
    ("sh*t", "shit"),
    ("s**t", "shit"),
    ("f*ck", "fuck"),
    ("fu*k", "fuck"),
    ("f**k", "fuck"),
    ("f*****g", "fucking"),
    ("f***ing", "fucking"),
    ("f**king", "fucking"),
    ("p*ssy", "pussy"),
    ("p***y", "pussy"),
    ("pu**y", "pussy"),
    ("p*ss", "piss"),
    ("b*tch", "bitch"),
    ("bit*h", "bitch"),
    ("h*ll", "hell"),
    ("h**l", "hell"),
    ("cr*p", "crap"),
    ("d*mn", "damn"),
    ("stu*pid", "stupid"),
    ("st*pid", "stupid"),
    ("n*gger", "nigger"),
    ("n***ga", "nigger"),
    ("f*ggot", "faggot"),
    ("scr*w", "screw"),
    ("pr*ck", "prick"),
    ("g*d", "god"),
    ("s*x", "sex"),
    ("a*s", "ass"),
    ("a**hole", "asshole"),
    ("a***ole", "asshole"),
    ("a**", "ass"),
    ("b*ll", "ball"),
    ("d*ck", "dick"),
    ("p*ssed", "pissed")
]

censored_replacer = [
    (re.compile(pat.replace("*", "\*"), flags=re.IGNORECASE), repl)
    for pat, repl in censored
]
RE_SPACE = re.compile(r"\s")
RE_MULTI_SPACE = re.compile(r"\s+")

SNOWBALL_STEMMER = SnowballStemmer("english")


def normalize(text: str) -> str:
    text = RE_SPACE.sub(" ", text)
    text = RE_MULTI_SPACE.sub(" ", text).strip()
    for pattern, repl in censored_replacer:
        text = pattern.sub(repl, text)
    
    new_list = []
    for word in text.split():
        new_list.append(SNOWBALL_STEMMER.stem(word))
    return " ".join(new_list)
    
import multiprocessing
with multiprocessing.Pool(processes=60) as pool:
     text_list = pool.map(normalize, df_sub.comment_text.tolist())

In [14]:
## Add cleaned text back to df
df['comment_text_clean'] = text_list

In [15]:
## Train/test split on normalized df

df_x_cl = np.array(df['comment_text_clean']).reshape(df['comment_text_clean'].shape[0],)
df_y_cl = np.array(df['label']).reshape(df['label'].shape[0],)
train_x_cl, test_x_cl, train_y_cl, test_y_cl = train_test_split(df_x_cl, df_y_cl, test_size = .3, random_state = 123)

In [ ]:
## GridSearchCV pipeline

tf_idf_pipe_cl = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', classifier_switch())
])

params = [
#     {'clf__estimator': [LogisticRegression(max_iter = 5000)],
#     'tfidf__stop_words': ['english', None],
#     'tfidf__strip_accents': ["ascii", "unicode", None]},
    {'clf__estimator': [LogisticRegression(max_iter = 5000)],
    'tfidf__stop_words': ['english', None],
    'clf__estimator__class_weight': [None, {0:1, 1:16}]},
#     {'clf__estimator': [LinearSVC()]},
#     {'clf__estimator': [LinearSVC(class_weight={0:1, 1:16})]},
    {'clf__estimator': [MultinomialNB()],
    'tfidf__stop_words': ['english', None],},
    {'clf__estimator': [MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)],
    'tfidf__stop_words': ['english', None],}
]

tf_idf_clf_cl = GridSearchCV(tf_idf_pipe_cl, params, refit="roc_auc", scoring = ["accuracy", 'precision', 'recall', "f1", 'roc_auc'], cv=5, n_jobs=-1, return_train_score=False, verbose=13)
tf_idf_clf_cl.fit(train_x_cl, train_y_cl)

In [ ]:
## Display metrics and calculate Bias AUC

preds = tf_idf_clf_cl.predict(test_x_cl)
validate_df = pd.DataFrame({"comment_text_clean": test_x_cl, "label": test_y, "lr": preds})
validate_df = validate_df.merge(df.drop(['label'], axis = 1), "inner", on = "comment_text_clean")
validate_df = convert_dataframe_to_bool(validate_df)
bias_metrics_df = compute_bias_metrics_for_model(validate_df, identity_columns, MODEL_NAME, TOXICITY_COLUMN)
print('Normal metrics for tf_idf model are: ')
print(pd.DataFrame(tf_idf_clf_cl.cv_results_)[["mean_test_accuracy","mean_test_precision", "mean_test_recall", "mean_test_f1", "mean_test_roc_auc"]])
print("Bias AUC Score for tf_idf model is: ", get_final_metric(bias_metrics_df, calculate_overall_auc(validate_df, MODEL_NAME)))